In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/01 19:40:25 WARN Utils: Your hostname, farik-ThinkPad-E14-Gen-2 resolves to a loopback address: 127.0.1.1; using 192.168.8.105 instead (on interface wlp0s20f3)
24/03/01 19:40:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/01 19:40:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [1]:
!wc -l fhvhv_tripdata_2021-01.csv

11908469 fhvhv_tripdata_2021-01.csv


In [5]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [7]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

In [8]:
!head -n 101 fhvhv_tripdata_2021-01.csv > head.csv

In [11]:
import pandas as pd

In [12]:
df_pandas = pd.read_csv("head.csv")

In [14]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [15]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [16]:
from pyspark.sql import types

In [17]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True), 
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True)
])

In [22]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [25]:
df.repartition(24)

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [26]:
df.write.parquet('fhvhv/2021/01/')

24/03/01 19:58:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [27]:
df = spark.read.parquet('fhvhv/2021/01/')

In [28]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [32]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003') \
    .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-17 06:20:53|2021-01-17 06:45:39|         148|         138|
|2021-01-17 06:27:43|2021-01-17 06:39:08|         231|          79|
|2021-01-17 06:44:48|2021-01-17 07:04:54|          79|         138|
|2021-01-17 06:48:16|2021-01-17 06:56:14|          14|         228|
|2021-01-17 06:08:05|2021-01-17 06:14:56|          73|         192|
|2021-01-17 06:33:25|2021-01-17 06:41:16|           7|         129|
|2021-01-17 06:46:40|2021-01-17 07:03:37|         129|          61|
|2021-01-17 06:27:04|2021-01-17 06:39:47|          21|          14|
|2021-01-17 06:42:44|2021-01-17 06:54:12|          14|          40|
|2021-01-17 06:08:53|2021-01-17 06:20:59|          42|         142|
|2021-01-17 06:26:38|2021-01-17 06:46:48|         238|         250|
|2021-01-17 06:16:39|2021-01-17 06:52:52|       

In [34]:
from pyspark.sql import functions as F

In [43]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [44]:
crazy_stuff('B02884')

's/b44'

In [45]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [48]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id','pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/b30| 2021-01-17|  2021-01-17|         148|         138|
|  s/b13| 2021-01-17|  2021-01-17|         231|          79|
|  s/b13| 2021-01-17|  2021-01-17|          79|         138|
|  e/acc| 2021-01-17|  2021-01-17|          14|         228|
|  e/acc| 2021-01-17|  2021-01-17|          73|         192|
|  e/acc| 2021-01-17|  2021-01-17|           7|         129|
|  e/acc| 2021-01-17|  2021-01-17|         129|          61|
|  e/b47| 2021-01-17|  2021-01-17|          21|          14|
|  e/b47| 2021-01-17|  2021-01-17|          14|          40|
|  a/a7a| 2021-01-17|  2021-01-17|          42|         142|
|  a/a7a| 2021-01-17|  2021-01-17|         238|         250|
|  e/9ce| 2021-01-17|  2021-01-17|         225|          61|
|  e/9ce| 2021-01-17|  2021-01-17|         225|         132|
|  s/acd| 2021-01-17|  2